In [1]:
import numpy as np
import pandas as pd

from keras.preprocessing import image
from keras_applications import imagenet_utils
#see doc for built-in functions https://github.com/keras-team/keras-applications/blob/master/keras_applications/imagenet_utils.py#L157
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.applications.imagenet_utils import decode_predictions

from keras.models import Model

from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.layers.convolutional import Conv2D, MaxPooling2D, ZeroPadding2D

Using TensorFlow backend.


In [2]:
import metis_passion_project.src.utils.preprocessing_functions  
import metis_passion_project.src.utils.prep_images

from metis_passion_project.src.utils.preprocessing_functions import get_images, to_array, cos_sim_vs_all, get_feature_df
from metis_passion_project.src.utils.prep_images import convert_image_to_bgr_numpy_array, prepare_image

import time

Instructions for updating:
Colocations handled automatically by placer.


In [3]:
VGG16_top = VGG16(weights='imagenet', include_top=True)
modelft = Model(inputs=VGG16_top.input, outputs=VGG16_top.get_layer('fc2').output)

In [4]:
modelft.summary()  #useful overview functions for model architecture!

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 3, 224, 224)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 64, 224, 224)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 64, 224, 224)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 64, 112, 112)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 128, 112, 112)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 128, 112, 112)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 128, 56, 56)       0   

In [5]:
img_path = 'testimages/dog.jpg'
img = image.load_img(img_path, target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

In [6]:
features = modelft.predict(x) #.predict is slow even for a single image

In [7]:
print(features.shape)
print(features) 

(1, 4096)
[[0.        0.        6.4011474 ... 3.977399  0.        0.       ]]


In [8]:
img_path = 'testimages/8301.jpg'
img = image.load_img(img_path, target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0) #?
x = preprocess_input(x) #what is this imported ftn doing?*
features = modelft.predict(x)

In [9]:
print(features) #get 0s on the border because image has a different size?

[[0. 0. 0. ... 0. 0. 0.]]


In [10]:
# def get_images(directory):
#     #loop through files in directory, directory is file path of folder containing images
#     import os
#     img_lst = []
#     for entry in os.scandir(directory): 
#         if entry.path == directory + '/.DS_Store':
#             continue  #avoid reading mac os's .DS_Store as an image
#         else:
#             img_lst.append(entry.path)
#     return img_lst

In [11]:
#from metis_passion_project.src.utils.preprocessing_functions import progressbar  *try importing again after restarting kernel?
import sys
def progressbar(it, prefix="", size=60, file=sys.stdout):
    count = len(it)
    def show(j):
        x = int(size*j/count)
        file.write("%s[%s%s] %i/%i\r" % (prefix, "#"*x, "."*(size-x), j, count))
        file.flush()        
    show(0)
    for i, item in enumerate(it):
        yield item
        show(i+1)
    file.write("\n")
    file.flush()
    

In [12]:
    
#for i in progressbar(range(20), "Computing: ", 40):
#    get_feature_index(array_list)

In [13]:
print(get_images('testimages/muffins'))

['testimages/muffins/19826.jpg', 'testimages/muffins/2945.jpg', 'testimages/muffins/6423.jpg', 'testimages/muffins/9186.jpg']


In [14]:
images_list = get_images('testimages/muffins')

In [15]:
array_list = to_array(images_list)

In [16]:
#VGG16_top = VGG16(weights='imagenet', include_top=True)  #also try inlude_top = False
#modelft = Model(inputs=VGG16_top.input, outputs=VGG16_top.get_layer('fc2').output)

In [17]:
# import pandas as pd
# feature_df = pd.DataFrame()   #make this into a function and export to .py**
# for im in array_list:
#     im = np.expand_dims(im, axis=0) #is this necessary?
#     im = preprocess_input(im) 
#     features = modelft.predict(im)#x?
#     feature_df = feature_df.append(pd.DataFrame(features), ignore_index=True)
    
#     #roughtly 4s for preprocessing 4 images and putting features in df

In [18]:
import pandas as pd
feature_df = pd.DataFrame()   #make this into a function and export to .py** -- this works better than imported ver somehow?
for im in array_list:
    im = np.expand_dims(im, axis=0) #is this necessary?
    im = preprocess_input(im) 
    features = modelft.predict(im)#x?
    feature_df = feature_df.append(pd.DataFrame(features), ignore_index=True)

    #add code printout to see how many images have been processed:
    #if im % 200 ==0:
     #       print("Done upto image {}".format(jpegImg)) doesn't work because im is array here

In [19]:
feature_df

,0,1,2,3,4,5,6,7,8,9,...,4086,4087,4088,4089,4090,4091,4092,4093,4094,4095
0,0.0,0.0,0.0,0.000000,0.482736,0.0,2.289247,0.0,0.0,0.000000,...,0.000000,0.0,0.0,0.466059,0.0,0.0,0.0,0.000000,0.0,0.0
1,0.0,0.0,0.0,0.144903,1.764754,0.0,5.023448,0.0,0.0,0.000000,...,0.000000,0.0,0.0,2.278557,0.0,0.0,0.0,0.000000,0.0,0.0
2,0.0,0.0,0.0,0.346135,0.000000,0.0,2.357480,0.0,0.0,0.000000,...,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0
3,0.0,0.0,0.0,0.000000,0.000000,0.0,1.343498,0.0,0.0,1.504429,...,0.439708,0.0,0.0,0.570876,0.0,0.0,0.0,3.334681,0.0,0.0


In [20]:
feature_df.values #get array of values 

array([[0.      , 0.      , 0.      , ..., 0.      , 0.      , 0.      ],
       [0.      , 0.      , 0.      , ..., 0.      , 0.      , 0.      ],
       [0.      , 0.      , 0.      , ..., 0.      , 0.      , 0.      ],
       [0.      , 0.      , 0.      , ..., 3.334681, 0.      , 0.      ]],
      dtype=float32)

In [21]:
#select a single row from df using .loc[]!
feature_df.iloc[3].values #2nd row (row index 1) 

array([0.      , 0.      , 0.      , ..., 3.334681, 0.      , 0.      ],
      dtype=float32)

In [22]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_similarity(feature_df.iloc[1].values.reshape(1,-1))  #*does have cos sim of 1 with itself! but self-defined function doesn't work

array([[1.]], dtype=float32)

In [23]:
# def cos_sim_vs_all(imageft, datasetft):
#     from sklearn.metrics.pairwise import cosine_similarity
#     sim_list = []
#     for i in range(len(datasetft)):
#         cos_sim = cosine_similarity(imageft, datasetft.iloc[i].values.reshape(1,-1)) 
#         sim_list.append(cos_sim)
#     return sim_list

In [24]:
cos_sim_vs_all(feature_df.iloc[1].values.reshape(1,-1),feature_df)  #works up to here

array([[[0.8342912]],

       [[1.       ]],

       [[0.8419672]],

       [[0.6739648]]], dtype=float32)

Muffin 1 is most similar with itself, quite similar to 0 and 2, least similar with 3 (#4 in list) - this makes sense to me visually also

---

In [25]:
#Nov 27, test new function to write csv
#load features from csv, to use as inputs for cos_sim_vs_all function
savetopath ='/Users/xinrucheng/Documents/Metis_bootcamp/week_9/metis_passion_project/data/processed/test.csv'

In [26]:
#get_images()
#to_array() -> array_list
#get_feature_df(array_list, savetopath)  #still ~4s for 4 images, not much faster than without csv?
#3s instead of 4 after commenting out preprocessing_input line in get_features

#11/27, 10pm 6s? taking longer? 2nd run, 3s

In [27]:
#test diff lines in get_feature - preprocessing code
#get_feature_df(array_list, savetopath)

In [28]:
# def get_feature_test(arraylist, savetopath): #combine with ftn above? keras vgg16 application example
#     '''use keras preprocessing functions to process each image and fit on pretrained model with imagenet weights
#     Returns a csv file from dataframes of features, 4096 features for each image (before last (top) layer of CNN model)
#     Modified from keras docs: https://keras.io/applications/#models-for-image-classification-with-weights-trained-on-imagenet
#     '''

#     #?

#     #*get rid of DF line, put array to csv directly?
#     #*use CNN lecture preprocessing code instead?
# #* break down preprocess ftn, see which line takes longest*!

#     feature_df = pd.DataFrame()   #*save as csv instead!** csv uses memory; df uses RAM
#     #how to loop through every 5 or 20 images for df? while loop?
#     for im in arraylist:
#         im = np.expand_dims(im, axis=0) #is this necessary?
#         im = preprocess_input(im) 
#         features = modelft.predict(im)#x?  #check keras docs? may not need this line
#         feature_df = pd.DataFrame(im) #df of one image at a time for now, optimize later
#         with open(savetopath, 'a') as f: #define mode as "a" . "A" stands for APPEND
#             feature_df.to_csv(f, mode='a', header=f.tell()==0) #add header when writing to file for the first time
#             #should this be in the for loop?

In [29]:
images_list = get_images('testimages/muffins') #4ms

In [30]:
array_list = to_array(images_list) #14ms for four images ~3ms per image
#restart kernel, 42ms? 13ms per image

In [31]:
get_feature_df(array_list, savetopath) 

(3, 224, 224)
(1, 3, 224, 224)


ValueError: Error when checking input: expected input_1 to have shape (224, 224, 3) but got array with shape (3, 224, 224)

In [ ]:
#**
#get_feature_test(array_list, savetopath) 
    #full function, 2.86s for four images ~0.72s/image; 2nd run: 3.28s ~0.82s/image
    #no expand_dims, get error ValueError: Error when checking input: expected input_3 to have 4 dimensions, but got array with shape (224, 224, 3)
    #without keras preprocessing function (just predict), 2.45, 2.47, 2.52s ~0.61~0.645s/image (still too slow?)

    #just process, no predict, 55ms ~14ms/image; 61ms ~15ms
    
    #14ms per image, 1000 images = 14s?
    
    
#from prev get_features function
#features = modelft.predict(im) -- without preprocessing ftn, just predict, get ~0.64ms/image
    #100 images, 64s (~1min); 2k images 1280 + ~130 for ~ 1400s = total ~20-30min without preproc ftn? but actually much slower, a lot of overhead (where)

In [ ]:
 #without keras preprocessing function, 2.45, 2.47, 2.52s ~0.61~0.645s/image (still too slow?)

#with preprocessing but without predict, dimensions incorrect - need all of them

In [ ]:
#read_csv: Read a comma-separated values (csv) file into DataFrame.
#instead of hardcoded paths? need to change this for aws code
muffin_ft = pd.read_csv('/Users/xinrucheng/Documents/Metis_bootcamp/week_9/metis_passion_project/data/processed/test.csv')

In [ ]:
muffin_ft #7 rows because ran twice, keeps appending onwards, and first row counted as header? 
#need to initialize file (in or out of function?)

In [ ]:
type(muffin_ft.iloc[0].values) #values of df Series values are in an array!

In [ ]:
cos_sim_vs_all(muffin_ft.iloc[0].values.reshape(1, -1), muffin_ft)  #use iloc for df indices!

Results make sense, muffin0 more similar to 1 and 2 compared to 3 (darker colour, flatter shape)

Can proceed with larger datasets now (finally)!

In [ ]:
muffins = cos_sim_vs_all(muffin_ft.iloc[0].values.reshape(1, -1), muffin_ft)

In [ ]:
type(muffins)

In [ ]:
#a = muffins.argsort() #don't use this
#argsort not working! but it's the correct data type; why?
#should return indices sorted by values, expect [3,1,2,0]

*write function for this also?*
argsort() Returns the indices that would sort an array. Default is quicksort

It returns an array of indices of the same shape as a that index data along the given axis in sorted order.

axis : [int or None] Axis along which to sort. If None, the array is flattened before sorting. The default is -1, which sorts along the last axis.

Example from numpy:
```
x = np.array([[0, 3], [2, 2]])

>>> ind = np.argsort(x, axis=1)  # sorts along last axis (across)
>>> ind
array([[0, 1],
       [0, 1]])
>>> np.take_along_axis(x, ind, axis=1)  # same as np.sort(x, axis=1)
array([[0, 3],
       [2, 2]])
       
       
       
>>> ind = np.argsort(x, axis=0)  # sorts along first axis (down)
>>> ind
array([[0, 1],
       [1, 0]])
>>> np.take_along_axis(x, ind, axis=0)  # same as np.sort(x, axis=0)
array([[0, 2],
       [2, 3]])
       ```
       
    

In [ ]:
a = muffins.argsort(axis=None)  #was probably sorting by some other axis, use this in .py function instead!
a 
#can connect image indicies to recipe titles later? or make image titles the indicies? - works for bbc set with recipe names as image title, but not for allr set where image titles are numbers

In [ ]:
#np.fliplr(a) #Requires the array to be at least 2-D! use it later

----

Take one pie photo, compare cosine similarity with the rest of the dataset

Datasets

allrecipes.com: 91K recipes, with photos, HTML, and crawling code
epicurious.com: 34K recipes, with photos, JSON, and crawling code
bbc.co.uk: 10K recipes, with photos, HTML, and crawling code
cookstr.com: 8K recipes, with photos, HTML, and crawling code -- need to change file extensions first

Wanted to start with bbc set, but not many pies. Previously got 0.75 with allrecipes data (largest, 35,300 items in 250x250 folderm ~same with epicurious set), Will likely merge all of them eventually anyway.

Try to find similar pictures in BBC set for: apple pie (perfect_apple_pie_73735_16x9.jpg) and roast turkey (perfect_roast_turkey_72482_16x9.jpg)

In [ ]:
#hardcode local path, better way?
#load_or_make.py, pickle data?
path = '/Users/xinrucheng/Documents/Metis_bootcamp/week_9/project5data/2017_140k/recipe_photos/bbc_photos/pages-photos'

In [ ]:
img_path = path + '/perfect_apple_pie_73735_16x9.jpg'
img = image.load_img(img_path, target_size=(224, 224))
x = image.img_to_array(img)

In [ ]:
x = np.expand_dims(x, axis=0) 

In [ ]:
x = preprocess_input(x) 

In [ ]:
fts = modelft.predict(x)

applepie_ft = fts

In [ ]:
type(applepie_ft)

In [ ]:
#split cells to see which line in get_features takes the longest
img_path = path + '/perfect_roast_turkey_72482_16x9.jpg'
img = image.load_img(img_path, target_size=(224, 224))  #put this in a ftn to loop through filenames

In [ ]:
x = image.img_to_array(img)

In [ ]:
x = np.expand_dims(x, axis=0) 

In [ ]:
x = preprocess_input(x) 

In [ ]:
fts1 = modelft.predict(x) #predict takes the longest, nothing I can do about it though*

In [ ]:
turkey_ft = fts1

In [ ]:
cosine_similarity(turkey_ft, applepie_ft) #inputs are arrays

In [ ]:
cosine_similarity(turkey_ft, turkey_ft)

In [ ]:
cosine_similarity(applepie_ft, applepie_ft) #makes sense features are most similar to itself

In [ ]:
# img_path = path + '/perfect_roast_turkey_72482_16x9.jpg'
# img = image.load_img(img_path, target_size=(224, 224))  #put this in a ftn to loop through filenames
# x = image.img_to_array(img)
# x = np.expand_dims(x, axis=0) 
# x = preprocess_input(x) 
# fts1 = modelft.predict(x)

# turkey_ft = fts1

In [ ]:
# def get_fta(arraylist): 
#     #from keras.applications.vgg16 import VGG16
#     #from keras.models import Model
#     #import pandas as pd
    
#     ft_arr = []
#     #VGG16_top = VGG16(weights='imagenet', include_top=True)
#     #modelft = Model(inputs=VGG16_top.input, outputs=VGG16_top.get_layer('fc2').output)
    
#     for im in arraylist:
#         #im = np.expand_dims(im, axis=0) #is this necessary?
#         im = preprocess_input(im)  #built-in, scales image also?
#         features = modelft.predict(im)
#         ft_arr = ft_arr.append(features)
#     return np.array(ft_arr)


In [ ]:
im.shape #does have 4 dimensions, maybe im needs to be returned?

In [ ]:
features.shape

In [ ]:
#cos_sim_vs_all(muf_df.iloc[1].values.reshape(1,-1),muf_df)  #diff data type?  1 array or list of 4 arrays

-- now getting different values compared to before? what changed? get_feature_df function, but should be identical in jupyter or imported!
```
[array([[0.79974574]], dtype=float32),
 array([[1.]], dtype=float32),
 array([[0.8104253]], dtype=float32),
 array([[0.6055574]], dtype=float32)]
 ```

In [ ]:
#get_ft_df(test2) #16s for 4 images, slower than before?
# when importing ftn or on aws machine: "ValueError: Error when checking input: expected input_3 to have 4 dimensions, but got array with shape (1, 1, 4096)"

#why 4 instead of 2 rows? not repeated from above either; already restarted jupyter

    #np.resize(img, (-1, <image shape>) #*need to reshape arrays too?
    
#np.resize(img, (-1, 224,224,3)  #*which input is expected to have 4 dimensions? input_n in error message keeps changing

#Nov 26 new error: 
#ValueError: Error when checking input: expected input_4 to have 4 dimensions, but got array with shape (1, 4096)



In [ ]:
# # from above
# def cos_sim_vs_all(imageft, datasetft):
#     from sklearn.metrics.pairwise import cosine_similarity
#     sim_list = []
#     for i in range(len(datasetft)):
#         cos_sim = cosine_similarity(imageft, datasetft.loc[i].values.reshape(1,-1)) #change to iloc?
#         #datasetft needs to be df for this to work - use output from get_ft_df
#             #better if take in array and matrix?
#         sim_list.append(cos_sim)
#     return sim_list


In [ ]:
#cos_sim_vs_all(turkey_ft, test2)

In [ ]:
#cos_sim_vs_all(turkey_ft, applepie_ft)

In [ ]:
#cos_sim_vs_all(turkey_ft, turkey_ft)

In [ ]:
#for a single image, results make sense (most similar to itself, cos sim~1! Something wrong with looping code, or uploading/importing to aws?

----

In [ ]:
#hardcode local path, better way?
#load_or_make.py, pickle data?
bbcpath = '/Users/xinrucheng/Documents/Metis_bootcamp/week_9/project5data/2017_140k/recipe_photos/bbc_photos/pages-photos'

In [ ]:
#bbcpath has 2226 images

In [ ]:
test_bbc = get_images(bbcpath)

In [ ]:
for i in progressbar(range(20), "Computing: ", 40):
    array_list = to_array(test_bbc) #using over 7GB RAM at 11/20 - activity monitor goes red* - save arrays of images as csv also?*
    
    #with more images, saving to csv is much faster (yay), ~2 min for 4/20 => 10 min for all? actually took 5mins
    
    #after removing preprocessing line, to_array is still ~ 5 mins

In [ ]:
#VGG16_top = VGG16(weights='imagenet', include_top=True)
#modelft = Model(inputs=VGG16_top.input, outputs=VGG16_top.get_layer('fc2').output)

In [ ]:
savetopath ='/Users/xinrucheng/Documents/Metis_bootcamp/week_9/metis_passion_project/data/processed/bbcft.csv'

In [ ]:
for i in progressbar(range(20), "Computing: ", 40):
    bbc_df = get_feature_df(array_list, savetopath) 
    #Nov 27 14:27, trying again with updated ftns, saving df to csv (saved to disk) this time, less pressure on RAM
    #doesn't seem that much faster initially, but memory pressure on activity monitor is green at least
    
    #still taking too long? 1/20 ~ 20-30mins, total could be  on local machine, try AWS again
    
    #started with new get_features ftn at 16:10 
        #after appending each image to csv, still slow! how to grab multiple rows in df to csv?
        
    #moved csv to inside for loop in get_features, commented out preprocessing_input, started this cell at 16:44
    #if 1s per image, 1/20~100 images should take <2mins 

In [ ]:
#first 1000 of bbc set? also try first 1000 of allrecipes set
#30mins for 1000 pictures on local machine   #afternoon: took 50mins on local machine??***

#started again at 19:36, only 7/20 done by 11pm; 
#aborted and restarted, finished 6:30am - ~7hrs, why so slow?? (kernel dies on aws)
#actually ~7hours for entire bbc images data, 2226 images*

In [ ]:
#check keras doc/blog? maybe use include_top = False instead in model?*

In [ ]:
img_path = path + '/perfect_roast_turkey_72482_16x9.jpg'
img = image.load_img(img_path, target_size=(224, 224))  #put this in a ftn to loop through filenames
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0) 
x = preprocess_input(x) 
fts1 = modelft.predict(x)

turkey_ft = pd.DataFrame(fts1)  #prev, array, change to df to feed into cos_sim_vs_all?

In [ ]:
turkey_ft.shape

In [ ]:
for i in progressbar(range(20), "Computing: ", 40):
    cos_sim_vs_all(turkey_ft,turkey_ft)  #similarity with itself is 1!
cos_sim_vs_all(turkey_ft,turkey_ft)

In [ ]:
#need to read out bbcfeatures csv before comparing! (also shouldn't call it df anymore)
#can write another function?
bbc_ft = pd.read_csv('/Users/xinrucheng/Documents/Metis_bootcamp/week_9/metis_passion_project/data/processed/bbcft.csv')

In [ ]:
#change function to save similarities as df instead of array?
for i in progressbar(range(20), "Computing: ", 40):
    
    cos_sim_vs_all(turkey_ft,bbc_df)  
cos_sim_vs_all(turkey_ft,bbc_df)
    #low similarity, but weird sorting prob is gone in similarity results
#choose something in first 200 images to compare against? bakewell tart
#should be able to use argsort on this?* got error earlier but was probably problem with cos_sim ftn itself?

---

In [ ]:
#keep list above, change to array, might get argsort working to see most similar pics**
#turkey_ar = np.array(turkey_sim)

In [ ]:
img_path = bbcpath + '/bakewelltart_89618_16x9.jpg'
img = image.load_img(img_path, target_size=(224, 224)) 
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0) 
x = preprocess_input(x) 
fts1 = modelft.predict(x)
#keras applications doc; similar to what get_features does

tart_ft = pd.DataFrame(fts1) 

In [ ]:
for i in progressbar(range(20), "Computing: ", 40):
    cos_sim_vs_all(tart_ft,bbc_df)  
cos_sim_vs_all(tart_ft,bbc_df)

In [ ]:
# for i in progressbar(range(20), "Computing: ", 40):
#     cos_sim_vs_all(turkey_ft.values.reshape(1,-1),bbc_df) #get same output as above, reshape not necessary - not throwing an error

In [ ]:
for i in progressbar(range(20), "Computing: ", 40):
    tart_sim = cos_sim_vs_all(tart_ft,tart_ft) 
#turkey_sim #.argsort() only works for arrays!

In [ ]:
type(tart_sim) #want an array for sorting

> argsort() documentations: Input array - arr : [array_like] 
Return : [index_array, ndarray] Array of indices that sort arr along the specified axis.If arr is one-dimensional then arr[index_array] returns a sorted arr.

In [ ]:
turkey_ar = np.array(turkey_sim)

In [ ]:
a = turkey_ar.argsort() #*need cos_sim features as a dataframe? 
#df of get_features not indexed?

In [ ]:
#np.fliplr(a)

In [ ]:
#change to array and argsort?

#reverse order
#argsort returns sorted array pf indices with their values from low to high 
# lsa_s_choc_ctv.argsort()
# Out[65]:
# array([[80276, 98585, 63735, ...,  5450, 35999,     3]])
# In [66]:
# a = lsa_s_choc_ctv.argsort()
# np.fliplr(a) #flip array sorted by argsort, slower though

# #expect the first result to be the recipe itself [3], with cosine similarity=1
# Out[66]:
# array([[    3, 35999,  5450, ..., 63735, 98585, 80276]])

In [ ]:
# def get_feature_index(arraylist):
#     '''use keras preprocessing functions to process each image and fit on pretrained model with imagenet weights
#     returns a dataframe of features, 4096 features for each image (before last (top) layer of CNN model)'''
#     feature_df = pd.DataFrame()   #save as csv instead?
#     for im in arraylist:
#         im = np.expand_dims(im, axis=0) #is this necessary?
#         im = preprocess_input(im) 
#         #print(im.shape)
#         features = modelft.predict(im)#x?
#         feature_df = feature_df.append(pd.DataFrame(features), ignore_index=False)
#     return feature_df  #(crashed on typo above, didn't run)

In [ ]:
#try enumerate?
# def cos_sim(imageft, datasetft):
#     '''Find the pairwise cosine similarity between features of the chosen image and all the images in the dataset'''
#     sim_list = []
#     for i in enumerate(datasetft):
#         cos_sim = cosine_similarity(imageft, datasetft.iloc[i].reshape(1,-1))
#         sim_list.append(cos_sim)
#     return np.array(sim_list)

In [ ]:
#testmuffin = cos_sim(muffin_df.iloc[0], muffin_df) #AttributeError: 'numpy.float32' object has no attribute 'values'?

In [ ]:
#turkey_sim = cos_sim_vs_all(muffin_df.iloc[0].values.reshape(1,-1),muffin_df) #throws error here if don't do reshape for one row
#new dim error on df, but comparing each element with the first muffin?

----
Test a portion of allrecipes images instead (more similar? but total of 35000 images is too big for now)

In [ ]:
arpath = '/Users/xinrucheng/Documents/Metis_bootcamp/week_9/project5data/2017_140k/recipe_photos/allre_images/userphotos/250x250'


In [ ]:
#**ran on AWS, took longer than 2 hours for get_features for first 1000 of allrecipes set

In [ ]:
#base_model = VGG16(weights='imagenet', include_top=False, input_shape=(3,250,250)) #include_top=False - dropped final layer of base model

In [ ]:
#using keras imagenet_utils functions to preprocess images
'''
def preprocess_input(x, data_format=None, mode='caffe', **kwargs):
    #Preprocesses a tensor or Numpy array encoding a batch of images.
    # Arguments
        x: Input Numpy or symbolic tensor, 3D or 4D.
            The preprocessed data is written over the input data
            if the data types are compatible. To avoid this
            behaviour, `numpy.copy(x)` can be used.
        data_format: Data format of the image tensor/array.
        mode: One of "caffe", "tf" or "torch".
            - caffe: will convert the images from RGB to BGR,
                then will zero-center each color channel with
                respect to the ImageNet dataset,
                without scaling. -> VGG16
            - tf: will scale pixels between -1 and 1,
                sample-wise.
            - torch: will scale pixels between 0 and 1 and then
                will normalize each channel with respect to the
                ImageNet dataset.
    # Returns
        Preprocessed tensor or Numpy array.
'''

In [ ]:
#feature_list = gen_features(array_list) #got similar error in test_CNN, dim3 instead of 2 for cos sim, how to fix?

Which preprocessing functions to use?
https://stackoverflow.com/questions/48677128/what-is-the-right-way-to-preprocess-images-in-keras-while-fine-tuning-pre-traine?rq=1

The input to applications.*.preprocess_input is always RGB. If a model expects BGR input, the channels will be permuted inside preprocess_input.